In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf().setMaster("local").setAppName("category-review-average")
sc = SparkContext(conf=conf)
# 초기화 

22/09/18 22:22:21 WARN Utils: Your hostname, Moon-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.4 instead (on interface en0)
22/09/18 22:22:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/18 22:22:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
directory = "/Users/sig6774/Desktop/Data_Engineering/data-engineering-main/01-spark/data"
filename = "restaurant_reviews.csv"

In [4]:
lines = sc.textFile(f"file:///{directory}/{filename}")

In [5]:
lines.collect()
# lines 객체에 들어있는 정보를 볼 수 있는 함수 

['id,item,cateogry,reviews',
 '0,짜장면,중식,125',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223   ',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [9]:
header = lines.first()
# 헤더 추출할 수 있음 

In [10]:
filtered_lines = lines.filter(lambda row:row != header)

In [11]:
filtered_lines.collect()

['0,짜장면,중식,125',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223   ',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [13]:
def parse(row):
    # '0,짜장면,중식,125'
    fields = row.split(",")
    category = fields[2]
    reviews = int(fields[3])
    
    return (category, reviews)
# 파싱할 수 있는 함수 구현 

In [14]:
categoryReviews = filtered_lines.map(parse)

In [15]:
categoryReviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [16]:
categoryReviewsCount = categoryReviews.mapValues(lambda x : (x,1))

In [17]:
categoryReviewsCount.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [18]:
reduced = categoryReviewsCount.reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))
# reduceByKey : 키를 기준으로 value를 task에 맞게 처리 
# 키를 기준으로 125와 같이 앞의 값을 더해줌, 키를 기준으로 1와 같이 뒤의 값을 더해줌 

In [19]:
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [20]:
averages = reduced.mapValues(lambda x: x[0] / x[1])
# mapValues()는 key가 변하지 않을 때 사용하면 좋음 
# key를 기준으로 tuple 형태인 value의 통계를 구할 수 있음 
averages.collect()
# 평균값 출력 가능 

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]